In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.dates as mdates
import os
import sys
import time
import scipy.signal


In [11]:
rf_raw=pd.read_csv('reflowoven_vaf_jan2.csv', usecols =['timestamp', 'data.A1', 'data.A2', 'data.A3'])

In [12]:
rf_raw.timestamp=pd.to_datetime(rf_raw.timestamp)
rf_raw.index=rf_raw.timestamp

In [13]:
rf_raw['total_current']=rf_raw['data.A1']+rf_raw['data.A2']+rf_raw['data.A3']

In [14]:
rf_raw['rolling_mean_p1']=pd.Series.to_frame(rf_raw['data.A1'].rolling(10, center=True).mean())

In [15]:
maxcur=62.0
rf_raw['state']=0

In [16]:
rf_raw['state'][rf_raw['rolling_mean_p1']<=1.5]=0
rf_raw['state'][rf_raw['rolling_mean_p1']>= 35]=2
rf_raw['state'][(rf_raw['rolling_mean_p1']>1.5) & (rf_raw['rolling_mean_p1']<=35)]=1

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports unti

In [17]:
%matplotlib qt

In [18]:
rf_raw.ix[0, 'state']=(rf_raw.ix[0, 'state']+1)%3
rf_raw.ix[-1, 'state']=(rf_raw.ix[0, 'state']+1)%3

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [19]:
plt.plot(rf_raw.timestamp, rf_raw.rolling_mean_p1)
plt.plot(rf_raw.timestamp, rf_raw.state*100)
plt.plot(rf_raw.total_current)

In [20]:
heating_times=scipy.signal.find_peaks(rf_raw.state, height=1.5, width=1)
heating_times_df=pd.DataFrame({"sample_number":heating_times[0], "working_time":heating_times[1]['widths']})

In [21]:
idle_temp=rf_raw[['state']]*-1
idle_times=scipy.signal.find_peaks(idle_temp.state, height=-.5, width=1)
idle_times_df=pd.DataFrame({"sample_number":idle_times[0], "working_time":idle_times[1]['widths']})

In [22]:
maintain_temp=rf_raw[['state']]
maintain_temp['state'][rf_raw.state==2]=0
maintain_times=scipy.signal.find_peaks(maintain_temp.state, height=.5, width=1)
maintain_times_df=pd.DataFrame({"sample_number":maintain_times[0], "working_time":maintain_times[1]['widths']})

/home/raghu/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/raghu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
rf_raw[['state']].plot()

In [24]:
for x, row in heating_times_df.iterrows():
    heating_times_df.ix[x,'start_time']=rf_raw.iloc[int(heating_times[1]['left_ips'][x])].timestamp
    heating_times_df.ix[x,'end_time']=rf_raw.iloc[int(heating_times[1]['right_ips'][x])].timestamp
    heating_times_df.ix[x,'timestamp']=rf_raw.iloc[int(heating_times[0][x])].timestamp
    heating_times_df.ix[x,'state']=rf_raw.iloc[int(heating_times[0][x])].state
    
for x, row in maintain_times_df.iterrows():
    maintain_times_df.ix[x,'start_time']=rf_raw.iloc[int(maintain_times[1]['left_ips'][x])].timestamp
    maintain_times_df.ix[x,'end_time']=rf_raw.iloc[int(maintain_times[1]['right_ips'][x])].timestamp
    maintain_times_df.ix[x,'timestamp']=rf_raw.iloc[int(maintain_times[0][x])].timestamp
    maintain_times_df.ix[x,'state']=rf_raw.iloc[int(maintain_times[0][x])].state
    
for x, row in idle_times_df.iterrows():
    idle_times_df.ix[x,'start_time']=rf_raw.iloc[int(idle_times[1]['left_ips'][x])].timestamp
    idle_times_df.ix[x,'end_time']=rf_raw.iloc[int(idle_times[1]['right_ips'][x])].timestamp
    idle_times_df.ix[x,'timestamp']=rf_raw.iloc[int(idle_times[0][x])].timestamp
    idle_times_df.ix[x,'state']=rf_raw.iloc[int(idle_times[0][x])].state

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after remo

In [25]:
try:
    heating_times_df['energy'] = heating_times_df.apply(lambda x: rf_raw.loc[(rf_raw.timestamp <= x.end_time) & 
                                                            (x.start_time <= rf_raw.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)
except:
    print('No heating.')
try:
    maintain_times_df['energy'] = maintain_times_df.apply(lambda x: rf_raw.loc[(rf_raw.timestamp <= x.end_time) & 
                                                            (x.start_time <= rf_raw.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)
except:
    print('No maintain.')

try:
    idle_times_df['energy'] = idle_times_df.apply(lambda x: rf_raw.loc[(rf_raw.timestamp <= x.end_time) & 
                                                            (x.start_time <= rf_raw.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)
except:
    print('No off.')

In [26]:
RF_states=pd.DataFrame()

In [27]:
try:
    RF_states=RF_states.append(heating_times_df[['timestamp', 'state', 'working_time', 'energy']])
except:
    print()
try:
    RF_states=RF_states.append(maintain_times_df[['timestamp', 'state', 'working_time', 'energy']])
except:
    print()
try:
    RF_states=RF_states.append(idle_times_df[['timestamp', 'state', 'working_time', 'energy']])
except:
    print()

In [28]:
RF_states.index=RF_states.timestamp
# RF_states.drop('timestamp', axis=1, inplace=True)
# RF_states

In [36]:
RF_states[RF_states['state']==0]

,timestamp,state,working_time,energy
timestamp,,,,
2019-01-01 22:46:59.736400,2019-01-01 22:46:59.736400,0.0,27914.0,1.680227
2019-01-02 18:18:27.391260,2019-01-02 18:18:27.391260,0.0,1251.5,0.074954
